In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
Wcap = 2254.4*1000 # 风电场容量、kw作为基值的标幺单位
Ccap = 29983520.0*0.4*0.3 # 百分之10的配储（kwh）
Pcmax = 0.407 # 1c的储能（kw）
Soc0 = 0.5
Socmax = 0.97
Socmin = 0.03
eta = 0.932
p = 0.453 
deltaT = 5/60 # 单个决策点放电时间为5min尺度
points = int(24*60/5) # 5min优化点的个数，该数值会随着时间尺度的缩小而增大，例如1min时间尺度的数据是1440个点
threshold = 50*1000/Wcap # 因为使用了非线性求解器，需要多留一点裕度
A = 1
N = 1
step = Wcap*0.1 # 储能免考核求解步长

# 初始化参数

param_dic = {
    "Wcap": Wcap,  # 风电场容量，单位 kW
    "Ccap": Ccap,  # 配储容量，单位 kWh
    "Pcmax": Pcmax,  # 储能最大充电功率，单位 kW
    "Soc0": Soc0,  # 初始储能状态（SOC）
    "Socmax": Socmax,  # 储能最大SOC
    "Socmin": Socmin,  # 储能最小SOC
    "eta": eta,  # 储能充放电效率
    "p": p,  # 电价
    "deltaT": deltaT,  # 决策点放电时间
    "points": points,  # 优化点的个数
    "threshold": threshold,  # 求解器裕度阈值
    "A":A,
    "N":N, # 计算的时间尺度（天）
    "step":step,

}
df = pd.DataFrame(pd.read_csv('std_data_5min.csv'))
PReal = pd.DataFrame(pd.read_csv('std_data_5min.csv'))["std_realpower"]
Pust = pd.DataFrame(pd.read_csv('std_data_5min.csv'))["std_ust_predpower"]


# 构造10min考核点
# 由5min构造10min考核点时，由于是后向平均，需要用i点和i点前一个数据求和除以二
# 105119
PReal = np.array(PReal)  # 确保PReal是一个numpy数组
averagesPReal = (PReal[:-1] + PReal[1:]) / 2.0
averagesPReal = np.append(averagesPReal, averagesPReal[-1]) # 保持数组长度一致

Pust = np.array(Pust)  # 确保PReal是一个numpy数组
averagesPPust = (Pust[:-1] + Pust[1:]) / 2.0
averagesPPust = np.append(averagesPPust, averagesPPust[-1]) # 保持数组长度一致


In [4]:
# 运行模拟
Pc_result = []
Pc_dp = 0 # 储能动作动态数组，为上一时刻计算的本时刻动作值
PReal_dp = 0 # 真实功率的动态数组，注意是修改后的
from tqdm import tqdm


squ = averagesPReal.copy()  # 使用.copy()方法来创建一个副本 # 认为实际值就是日前的短期预测，也就是预测准确率是100%

for i in tqdm(range(len(averagesPReal) - 16)):
    
    # 创建一个16长度的变量数组，每个元素代表对应点的调整量
    optim_num = 16
    adjustments = cp.Variable(optim_num)

    # 目标是最小化所有调整量的平方和
    objective = cp.Minimize(cp.sum_squares(adjustments))

    # 假设averagesPPust的长度至少为 i + 16
    constraints = [cp.abs(squ[i+j-5] + adjustments[j] - squ[i+j+2-5] - adjustments[j+2]) <= 0.022
                for j in range(0, optim_num-2)] # 14是因为我们需要保证j+2不超过15，保持在adjustments的索引范围内
    # 定义问题并求解
    problem = cp.Problem(objective, constraints)
    problem.solve()
    Pc_ust = adjustments.value
    Pc_dp = Pc_ust[1+5] # 下一个决策点的功率动作值
    # 注意，调整后的功率必须作为下一次决策的功率
    squ[i+1] = Pc_dp + averagesPReal[i+1] # 对下一个点做调控，于是下一个点会受到影响
    Pc_result.append(Pc_dp)
    

    

  0%|          | 0/105102 [00:00<?, ?it/s]

 94%|█████████▍| 98808/105102 [39:48<02:25, 43.26it/s] 

In [ ]:

def calc_Ccap(PReal,param_dic,Pc_thy):
    
    '''
    计算功率变化在N天时间尺度下，能够完全免考核的储能的能量值和功率值
    :param A: 风电功率波动考核系数
    :param PReal: 风电功率实际值
    :param Wcap: 风机容量 kw
    :param p: 考核电价 元/kw
    :param threshold: 波动率要求
    :return Pc_thy: 输出不考虑能量约束和功率约束的Pc的理论值
    :return Pc: 输出考虑能量约束和功率约束的Pc的实际值
    :return penalty_fee: 计算单日考核费用的函数（按日考核）
    '''
    # 初始化参数
    Socmax = param_dic['Socmax']
    Socmin = param_dic['Socmin']
    Wcap = param_dic["Wcap"]
    Ccap = param_dic["Ccap"]
    Soc0 = param_dic["Soc0"] # 储能的初始值，通常取0.5
    deltaT = param_dic["deltaT"]
    eta = param_dic["eta"]
    step = param_dic["step"]
    max_iter = 1e9

    # 注意：由于deltaP数组长度比PReal短2，因此Pc_thy的最后两个元素将保持为0
    # 保存最大和最小的那个Pc[i]，作为电池功率放电上限和充电上限Pc_max,Pd_max
    # 找出最大的正值作为放电上限Pd_max
    Pd_max = np.max(Pc_thy)
    # 找出最小的值（如果有充电操作，应为负值），然后取其绝对值作为充电上限Pc_max
    Pc_max = abs(np.min(Pc_thy))

    # 但是由于储能容量（能量）上限与储能能量初始值相关，所以无法计算得出，但可以通过网格搜索确定在一个确定初始能量的情况下，恰好满足免考核的的储能容量是多少。
    # 定义一个波动总和阈值，当储能容量使得功率波动总和小于这个阈值，输出这个储能容量，作为电池容量上限。
    iter = 0
    while True:
        # 储能按照Pc_thy序列动作，但是需要考虑能量约束
        # 开始计算电池容量是否满足序列Pc_thy
        Soc = Soc0
        Soc_result = [Soc0] # 可视化Soc变化
        for i in range(0,len(Pc_thy)):
            # 能量状态转移方程：
            Soc += (np.maximum(-Pc_thy[i], 0) * eta - np.maximum(Pc_thy[i], 0) / eta) * deltaT / Ccap * Wcap
            Soc_result.append(Soc)
            flag = 1
            if Soc > Socmax or Soc < Socmin: # 如果能量约束被打破，则说明电池容量不合理，需要增大。
                Ccap += step # 增大储能，并重新开始计算
                print(Ccap)
                iter += 1
                # 清空Soc_result列表
                Soc_result.clear()
                flag = 0
                break
        if iter == max_iter:
            print("reach max iter:",max_iter)
            return Ccap,Pc_max,Pd_max,Pc_thy,Soc_result
        elif flag == 1:
            print("solv found")
            return Ccap,Pc_max,Pd_max,Pc_thy,Soc_result

In [ ]:
n = len(Pc_result)
averagesPReal[:n]
averagesPPust[:n]
ad_real = Pc_result + averagesPReal[:n]
Ccap,Pc_max,Pd_max,Pc_thy,Soc_result = calc_Ccap(PReal,param_dic,Pc_result)


import plotly.graph_objs as go

# 创建图表数据
trace1 = go.Scatter(x=list(range(len(Pc_result))), y=Pc_result, mode='lines', name='Pc_result')
trace2 = go.Scatter(x=list(range(len(averagesPReal[:n]))), y=averagesPReal[:n], mode='lines', name='averagesPReal')
trace3 = go.Scatter(x=list(range(len(averagesPPust[:n]))), y=averagesPPust[:n], mode='lines', name='averagesPPust')
trace4 = go.Scatter(x=list(range(len(Pc_result))), y=[Pc_result[i] + averagesPReal[i] for i in range(len(Pc_result))], mode='lines', name='ad_real')
trace5 = go.Scatter(x=list(range(len(Soc_result))), y=Soc_result, mode='lines', name='Soc_result')

# 组合图表数据
data = [trace1, trace2, trace3, trace4,trace5]

# 创建布局
layout = go.Layout(title='四个变量的折线图',
                   xaxis=dict(title='索引'),
                   yaxis=dict(title='值'))

# 创建图表
fig = go.Figure(data=data, layout=layout)

# 显示图表
fig.show()


In [ ]:

# Pc_result 动作序列最少，会比实际序列少16个，因为滑动窗口不够长了。Soc会比Pc_result多一个，因为Soc是积分
# 创建一个DataFrame
df = pd.DataFrame({
    'Pc_result': Pc_result,
    'averagesPReal': averagesPReal[:len(Pc_result)],
    'Soc_result':Soc_result[:len(Pc_result)],
    'averagesPPust':averagesPPust[:len(Pc_result)]
})
# 将DataFrame保存为CSV文件
csv_file_path = 'std_ust_optim.csv'
df.to_csv(csv_file_path, index=False)
print('附录F（1）基于滚动优化动作曲线的功率平滑方案代码。Pc_result为仅仅应用了滚动方案而没有应用充电机会方案的储能动作值；averagesPReal，averagesPPred为实际值和超短期预测的10分钟后向平均；Soc_result为仅仅应用了滚动方案而没有应用充电机会方案的储能Soc值。')

In [ ]:
# 计算相邻10分钟的功率波动
fluctuations = []
for i in range(0, len(averagesPReal[:n]) - 2):  # 减2是因为我们是计算相隔10分钟（即每隔一个索引）的差值
    fluctuation = abs(averagesPReal[:n][i] - averagesPReal[:n][i+2])
    fluctuations.append(fluctuation)
sum_exceed_threshold = sum(value for value in fluctuations if value > threshold)

penalty = A*sum_exceed_threshold*10/60*p*Wcap
print("调节前考核电量（标幺）：",sum_exceed_threshold)

In [ ]:
# 计算相邻10分钟的功率波动
fluctuations = []
for i in range(0, len(ad_real) - 2):  # 减2是因为我们是计算相隔10分钟（即每隔一个索引）的差值
    fluctuation = abs(ad_real[i] - ad_real[i+2])
    fluctuations.append(fluctuation)
sum_exceed_threshold = sum(value for value in fluctuations if value > threshold)

penalty = A*sum_exceed_threshold*10/60*p*Wcap
print("调节后考核电量（标幺）：",sum_exceed_threshold)